# Prepare Data for Modelling (Data Splits, including thresholding and PCA)
The current notebook is used to create datasets that can be used for modelling. Updated 21st of May!

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pickle as pkl
from sklearn import datasets
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD

# 1. Make the baseline data (with one-hot encoded categorical variables)

In [38]:
### TRYING TO DROP NA VALUES
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# load data
root = Path.cwd().parents[1]
baseline_file_path = root / "data" / "raw" / "healthy_subset_df_with_meta.csv"
demographic_data = pd.read_csv(baseline_file_path)

# calculate lactosum and create binary target variable 
lacto_columns = [col for col in demographic_data.columns if 'lactobacillus' in col.lower()]
demographic_data['LactoSum'] = demographic_data[lacto_columns].sum(axis=1)
demographic_data['Lacto_Binary'] = np.where(demographic_data['LactoSum'] > 0.01, 1, 0).astype(int)

# keep only columns containing the string 'Lacto_Binary' or columns that don't contain the string '|'
cols_to_keep = [col for col in demographic_data.columns if 'Lacto_Binary' in col or '|' not in col]
demographic_data = demographic_data[cols_to_keep]

# print values of Lacto_Binary to check, it should be 2776/2726
print(demographic_data['Lacto_Binary'].value_counts())
display(demographic_data.head())

# drop LactoSum column
demographic_data = demographic_data.drop(columns=['LactoSum'])

# save the cleaned data 
demographic_data.to_csv(root / "data" / "baseline_demographic" / "healthy_subset_df_with_meta_lactobinary.csv", index=False)

# ---- Partition data 
selected_variables = [
    'age', 'gender', 'country', 'BMI', 'diet', 'smoker', 'alcohol'
]

# drop all columns that are not 'Lacto_Binary' or in the selected_variables list
demographic_data = demographic_data[[col for col in demographic_data.columns if 'Lacto_Binary' in col or col in selected_variables]]

# One-hot encode categorical variables with 0/1 encoding
categorical_vars = ['gender', 'country', 'diet', 'smoker', 'alcohol']
demographic_data = pd.get_dummies(demographic_data, columns=categorical_vars, drop_first=True, dtype=int)

# Update selected_variables after one-hot encoding
encoded_variables = list(demographic_data.columns)
selected_variables = [var for var in encoded_variables if any(orig_var in var for orig_var in ['age', 'BMI', 'gender', 'country', 'diet', 'smoker', 'alcohol'])]

# Drop rows with any NaN values
demographic_data = demographic_data.dropna()

# split data into features (X) and target (y)
X = demographic_data[selected_variables]
y = demographic_data['Lacto_Binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, # 15% of data is reserved for testing
                                                    random_state=42)

# further splitting the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, # X_train is further split into X_train and X_val
                                                  y_train, # y_train is further split into y_train and y_val
                                                  test_size=X_test.shape[0] / X_train.shape[0],  # 15% of the training set is reserved for validation
                                                  random_state=42)

datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test} # create a dictionary with the datasets

# open a file to write
with open('../../data/baseline_demographic/dataset_distribution.txt', 'w') as file:
    # print table headers and write to file
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    # loop through each dataset, print proportions and total counts, and write to file
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)  # total number of observations
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')

for x, y, name in zip([X_train, X_val, X_test], 
                      [y_train, y_val, y_test], 
                      ['train', 'val', 'test']):
    combined_data = np.hstack([x, y.values.reshape(-1, 1)]) # combine X and y for each dataset 
    column_names = list(X.columns) + ['Lacto_Binary'] # construct list of column names, add Lacto_Binary to make sure it is the last column 
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(f'../../data/baseline_demographic/{name}.csv', index=False)


Lacto_Binary
0    2776
1    2726
Name: count, dtype: int64


/tmp/ipykernel_3008/2634217379.py:10: DtypeWarning: Columns (20,21,22,25,26,28,29,30,31,32,34,37,45,46,47,48,49,50,55,56,57,58,60,72,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  demographic_data = pd.read_csv(baseline_file_path)


,Unnamed: 0,sample_id,study_name,subject_id,body_site,antibiotics_current_use,study_condition,disease,age,age_category,...,fasting_glucose,ajcc,c_section_type,zigosity,brinkman_index,alcohol_numeric,ALT,eGFR,LactoSum,Lacto_Binary
0,1,a00820d6-7ae6-11e9-a106-68b59976a384,ShaoY_2019,B01042_mo,stool,NaN,control,healthy,32,adult,...,NaN,NaN,Elective_CS,NaN,NaN,NaN,NaN,NaN,0.00000,0
1,2,A01_02_1FE,PasolliE_2019,A01_02_1FE,stool,no,control,healthy,21,adult,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0
2,3,A02_01_1FE,PasolliE_2019,A02_01_1FE,stool,no,control,healthy,24,adult,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0
3,4,A03_01_1FE,PasolliE_2019,A03_01_1FE,stool,no,control,healthy,38,adult,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0
4,5,A04_01_1FE,PasolliE_2019,A04_01_1FE,stool,no,control,healthy,34,adult,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03775,1


Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3168
Validation set  | 0.47 | 0.53 |     680
Test set        | 0.49 | 0.51 |     680


In [39]:
# count how many rows in X_train has at least one 'NaN' value. 
nan_train = X_train.isnull().any(axis=1).sum()
print(f'Number of rows in X_train with at least one NaN value: {nan_train}')

nan_val = X_val.isnull().any(axis=1).sum()
print(f'Number of rows in X_val with at least one NaN value: {nan_val}')

nan_test = X_test.isnull().any(axis=1).sum()
print(f'Number of rows in X_test with at least one NaN value: {nan_test}')

print("-----------------------------------------------------------------------")
# print total number of rows in each dataset
print(f'Total number of rows in X_train: {X_train.shape[0]}')
print(f'Total number of rows in X_val: {X_val.shape[0]}')
print(f'Total number of rows in X_test: {X_test.shape[0]}')


print("-----------------------------------------------------------------------")
# The number of rows left in each dataset if I remove rows with at least one NaN value
left_train = X_train.dropna().shape[0]
left_val = X_val.dropna().shape[0]
left_test = X_test.dropna().shape[0]

print(f'Number of rows left in X_train if I remove rows with at least one NaN value: {left_train}')
print(f'Number of rows left in X_val if I remove rows with at least one NaN value: {left_val}')
print(f'Number of rows left in X_test if I remove rows with at least one NaN value: {left_test}')



Number of rows in X_train with at least one NaN value: 0
Number of rows in X_val with at least one NaN value: 0
Number of rows in X_test with at least one NaN value: 0
-----------------------------------------------------------------------
Total number of rows in X_train: 3168
Total number of rows in X_val: 680
Total number of rows in X_test: 680
-----------------------------------------------------------------------
Number of rows left in X_train if I remove rows with at least one NaN value: 3168
Number of rows left in X_val if I remove rows with at least one NaN value: 680
Number of rows left in X_test if I remove rows with at least one NaN value: 680


# 2. Make the thresholded data

In [40]:
root = Path.cwd().parents[1]
data_path = root / "data" / "raw" / "healthy_subset_df.csv"
data = pd.read_csv(data_path)
lacto_columns = [col for col in data.columns if 'lactobacillus' in col.lower()]
data['LactoSum'] = data[lacto_columns].sum(axis=1)
data['Lacto_Binary'] = np.where(data['LactoSum'] > 0.01, 1, 0).astype(int) # astype(int) converts True/False to 1/0 
# drop the LactoSum column
data = data.drop(columns=['LactoSum'])

# ------------- PARTITION TEMPORARY DATA (BEFORE THRESHOLDING)
X = data.drop(columns=['sample_id', 'Lacto_Binary'])
y = data['Lacto_Binary']


X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, # 15% of data is reserved for testing
                                                    random_state=42)

# further splitting the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, # X_train is further split into X_train and X_val
                                                  y_train, # y_train is further split into y_train and y_val
                                                  test_size=X_test.shape[0] / X_train.shape[0],  # 15% of the training set is reserved for validation
                                                  random_state=42)



# ------ FIGURING OUT WHICH COLS ARE BWLOW THRESHOLD ---------- # 
species_columns = [col for col in X_train.columns if '|' in col]
# calculate sum for each column
columns_sums = X_train[species_columns].sum()
# calculate total sum across all columns and rows
total_sum = columns_sums.sum()
# calculate percentage for each column
columns_percentages = (columns_sums / total_sum) * 100
# find columns where percentage is 0.1% or more
filtered_columns = columns_percentages[columns_percentages >= 0.1]

# filter X_train to keep only columns with percentage >= 0.1%
X_train_filtered = X_train[filtered_columns.index]

# Remove filtered columns from X_val and X_test
X_val_filtered = X_val[filtered_columns.index]
X_test_filtered = X_test[filtered_columns.index]


# ------ SAVE THE TRHESHOLDED DATA ------ # 
datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test} # create a dictionary with the datasets
output_dir = root / 'data' / 'reduced_0_1'
output_dir.mkdir(parents=True, exist_ok=True)

# open a file to write
with open(output_dir / 'dataset_distribution.txt', 'w') as file:
    # print table headers and write to file
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    # loop through each dataset, print proportions and total counts, and write to file
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)  # total number of observation
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')


for x, y, name in zip([X_train_filtered, X_val_filtered, X_test_filtered], [y_train, y_val, y_test], ['train', 'val', 'test']):
    combined_data = np.hstack([x, y.values.reshape(-1, 1)]) # combine X and y for each dataset 
    column_names = list(x.columns) + ['Lacto_Binary'] # construct list of column names, add Lacto_Binary to make sure it is the last column 
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(output_dir /f'{name}.csv', index=False)


Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3850
Validation set  | 0.51 | 0.49 |     826
Test set        | 0.48 | 0.52 |     826


# Thresholded + PCA 
I do this because the data is very sparse and has a lot of features. I want to reduce the number of features to make the model more interpretable and to reduce the risk of overfitting.
I do this by first thresholding the data and then applying PCA. PCA decrease the number of features by selecting dimension of features which have most of the variance. Data is still on the same scale and sum to 1, and therefore I have chosen not to scale the data before applying PCA. 


In [41]:
root = Path.cwd().parents[1]
data_path = root / "data" / "raw" / "healthy_subset_df.csv"
data = pd.read_csv(data_path)
lacto_columns = [col for col in data.columns if 'lactobacillus' in col.lower()]
data['LactoSum'] = data[lacto_columns].sum(axis=1)
data['Lacto_Binary'] = np.where(data['LactoSum'] > 0.01, 1, 0).astype(int) # astype(int) converts True/False to 1/0 
# drop the LactoSum column
data = data.drop(columns=['LactoSum'])



# ------------- PARTITION TEMPORARY DATA (BEFORE THRESHOLDING because thresholding should be done using only the training data)
X = data.drop(columns=['sample_id', 'Lacto_Binary'])
y = data['Lacto_Binary']


X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, # 15% of data is reserved for testing
                                                    random_state=42)

# further splitting the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, # X_train is further split into X_train and X_val
                                                  y_train, # y_train is further split into y_train and y_val
                                                  test_size=X_test.shape[0] / X_train.shape[0],  # 15% of the training set is reserved for validation
                                                  random_state=42)



# ------ FIGURING OUT WHICH COLS ARE BWLOW THRESHOLD ---------- # 
species_columns = [col for col in X_train.columns if '|' in col]
# calculate sum for each column
columns_sums = X_train[species_columns].sum()
# calculate total sum across all columns and rows
total_sum = columns_sums.sum()
# calculate percentage for each column
columns_percentages = (columns_sums / total_sum) * 100
# find columns where percentage is lower than 0.1% 
filtered_columns = columns_percentages[columns_percentages > 0.1]

# filter X_train to keep only columns with percentage > 0.1%
X_train_filtered = X_train[filtered_columns.index]

# Remove filtered columns from X_val and X_test
X_val_filtered = X_val[filtered_columns.index]
X_test_filtered = X_test[filtered_columns.index]


# ------- Perform PCA on the thresholded data ----- # 
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_filtered)
print(pca.explained_variance_ratio_)

# Project the validation and test sets onto the PCA space generated from the training data
X_val_pca = pca.transform(X_val_filtered)
X_test_pca = pca.transform(X_test_filtered)

# ------ SAVE THE TRHESHOLDED DATA ------ # 
datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test}
with open('../../data/reduced_0_1_PCA/dataset_distribution.txt', 'w') as file:
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')

for x_pca, y, name in zip([X_train_pca, X_val_pca, X_test_pca], [y_train, y_val, y_test], ['train', 'val', 'test']):
    combined_data = np.hstack([x_pca, y.values.reshape(-1, 1)])
    column_names = [f'PC{i+1}' for i in range(x_pca.shape[1])] + ['Lacto_Binary']
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(f'../../data/reduced_0_1_PCA/{name}.csv', index=False)


[0.31394337 0.09736937 0.05340137 0.04002439 0.03690475 0.03289054
 0.02757451 0.02682614 0.02580687 0.0253191 ]
Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3850
Validation set  | 0.51 | 0.49 |     826
Test set        | 0.48 | 0.52 |     826


# SVD Thresholded Data

In [44]:
root = Path.cwd().parents[1]
data_path = root / "data" / "raw" / "healthy_subset_df.csv"
data = pd.read_csv(data_path)
lacto_columns = [col for col in data.columns if 'lactobacillus' in col.lower()]
data['LactoSum'] = data[lacto_columns].sum(axis=1)
data['Lacto_Binary'] = np.where(data['LactoSum'] > 0.01, 1, 0).astype(int) # astype(int) converts True/False to 1/0 
# drop the LactoSum column
data = data.drop(columns=['LactoSum'])



# ------------- PARTITION TEMPORARY DATA (BEFORE THRESHOLDING because thresholding should be done using only the training data)
X = data.drop(columns=['sample_id','Lacto_Binary'])
y = data['Lacto_Binary']



X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, # 15% of data is reserved for testing
                                                    random_state=42)

# further splitting the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, # X_train is further split into X_train and X_val
                                                  y_train, # y_train is further split into y_train and y_val
                                                  test_size=X_test.shape[0] / X_train.shape[0],  # 15% of the training set is reserved for validation
                                                  random_state=42)


# ------ FIGURING OUT WHICH COLS ARE BWLOW THRESHOLD ---------- # 
species_columns = [col for col in X_train.columns if '|' in col]
# calculate sum for each column
columns_sums = X_train[species_columns].sum()
# calculate total sum across all columns and rows
total_sum = columns_sums.sum()
# calculate percentage for each column
columns_percentages = (columns_sums / total_sum) * 100
# find columns where percentage is lower than 0.1% 
filtered_columns = columns_percentages[columns_percentages > 0.1]

# filter X_train to keep only columns with percentage > 0.1%
X_train_filtered = X_train[filtered_columns.index]

# Remove filtered columns from X_val and X_test
X_val_filtered = X_val[filtered_columns.index]
X_test_filtered = X_test[filtered_columns.index]

# ------- Perform SVD on the thresholded data ----- # 
n_components = 10  # Fixed number of components to match PCA
svd = TruncatedSVD(n_components=n_components)
X_train_svd = svd.fit_transform(X_train_filtered)

# Project the validation and test sets onto the SVD space generated from the training data
X_val_svd = svd.transform(X_val_filtered)
X_test_svd = svd.transform(X_test_filtered)

# Print the explained variance ratio for consistency check
explained_variance_ratio = svd.explained_variance_ratio_
print(f"The components explain {explained_variance_ratio} variance")

# ------ SAVE THE TRHESHOLDED DATA ------ # 
datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test}
with open('../../data/reduced_0_1_SVD/dataset_distribution.txt', 'w') as file:
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')

for x_pca, y, name in zip([X_train_svd, X_val_svd, X_test_svd], [y_train, y_val, y_test], ['train', 'val', 'test']):
    combined_data = np.hstack([x_pca, y.values.reshape(-1, 1)])
    column_names = [f'PC{i+1}' for i in range(x_pca.shape[1])] + ['Lacto_Binary']
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(f'../../data/reduced_0_1_SVD/{name}.csv', index=False)


The components explain [0.24562669 0.09070127 0.0926026  0.05162535 0.03997536 0.03354777
 0.03274373 0.02753813 0.02651421 0.0256347 ] variance
Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3850
Validation set  | 0.51 | 0.49 |     826
Test set        | 0.48 | 0.52 |     826


# The Not-thresholded data

In [43]:
root = Path.cwd().parents[1]
data_path = root / "data" / "raw" / "healthy_subset_df.csv"
data = pd.read_csv(data_path)
lacto_columns = [col for col in data.columns if 'lactobacillus' in col.lower()]
data['LactoSum'] = data[lacto_columns].sum(axis=1)
data['Lacto_Binary'] = np.where(data['LactoSum'] > 0.01, 1, 0).astype(int) # astype(int) converts True/False to 1/0 
# drop the LactoSum column
data = data.drop(columns=['LactoSum'])


# ------------- PARTITION TEMPORARY DATA (BEFORE THRESHOLDING)
X = data.drop(columns=['Lacto_Binary'])
y = data['Lacto_Binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, # 15% of data is reserved for testing
                                                    random_state=42)

# further splitting the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, # X_train is further split into X_train and X_val
                                                  y_train, # y_train is further split into y_train and y_val
                                                  test_size=X_test.shape[0] / X_train.shape[0],  # 15% of the training set is reserved for validation
                                                  random_state=42)



# ------ SAVE THE TRHESHOLDED DATA ------ # 
datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test} # create a dictionary with the datasets

# open a file to write
with open('../../data/non_reduced/dataset_distribution.txt', 'w') as file:
    # print table headers and write to file
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    # loop through each dataset, print proportions and total counts, and write to file
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)  # total number of observation
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')


for x, y, name in zip([X_train, X_val, X_test], 
                      [y_train, y_val, y_test], 
                      ['train', 'val', 'test']):
    combined_data = np.hstack([x, y.values.reshape(-1, 1)]) # combine X and y for each dataset 
    column_names = list(X.columns) + ['Lacto_Binary'] # construct list of column names, add Lacto_Binary to make sure it is the last column 
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(f'../../data/non_reduced/{name}.csv', index=False)


Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3850
Validation set  | 0.51 | 0.49 |     826
Test set        | 0.48 | 0.52 |     826


# OLD SHITSHOW

## 0. Prepare baseline demographic data for analysis

In [145]:
# print the last column of the data frame to make sure that the Lacto_Binary column has been added
print(demographic_data.columns[-1])
# print first column to check it is the sampleID
print(demographic_data.columns[0])


Lacto_Binary
sample_id


In [146]:
demographic_data.to_csv('../../data/baseline_demographic/healthy_subset_df_with_meta_lactobinary.csv', index=False)

In [147]:
# specify the subset of demographic variables to use in baseline model
selected_variables = [
    'age', 'gender', 'country', 'BMI', 'diet', 'smoker', 'alcohol'
] 

# split data into features (X) and target (y)
X = demographic_data[selected_variables]
y = demographic_data['Lacto_Binary']  

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, # 15% of data is reserved for testing
                                                    random_state=42)

# further splitting the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, # X_train is further split into X_train and X_val
                                                  y_train, # y_train is further split into y_train and y_val
                                                  test_size=X_test.shape[0] / X_train.shape[0],  # 15% of the training set is reserved for validation
                                                  random_state=42)

In [149]:
datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test} # create a dictionary with the datasets

# open a file to write
with open('../../data/baseline_demographic/dataset_distribution.txt', 'w') as file:
    # print table headers and write to file
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    # loop through each dataset, print proportions and total counts, and write to file
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)  # total number of observation
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')

Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3850
Validation set  | 0.47 | 0.53 |     826
Test set        | 0.50 | 0.50 |     826


In [150]:
for x, y, name in zip([X_train, X_val, X_test], 
                      [y_train, y_val, y_test], 
                      ['train', 'val', 'test']):
    combined_data = np.hstack([x, y.values.reshape(-1, 1)]) # combine X and y for each dataset 
    column_names = list(X.columns) + ['Lacto_Binary'] # construct list of column names, add Lacto_Binary to make sure it is the last column 
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(f'../../data/baseline_demographic/{name}.csv', index=False)

## 1. Prepare the non-reduced data for analysis

In [151]:
# load data 
data_path = root / "data" / "raw" / "healthy_subset_df.csv"
data = pd.read_csv(data_path, index_col=0)

# check shape of data
print(f"Initial data shape: {data.shape}")

print("----------------------------------------------------------------------------------------------------")

# extract indices for columns that include the word 'lactobacillus' (looking at lower-cased col names)
lacto_index = [i for i, s in enumerate(data.columns) if 'lactobacillus' in s.lower()]

# for each row, calculate the sum of the Lactobacillus species and append value to new column 'LactoSum'
data['LactoSum'] = data.iloc[:, lacto_index].sum(axis=1) 

# count the number of rows that have a LactoSum value of above 0.01 and 0.0, respectively
print(f"There are {data[data['LactoSum'] > 0.0].shape[0]} samples that have a LactoSum value of above 0.0")
print(f"There are {data[data['LactoSum'] > 0.01].shape[0]} samples that have a LactoSum value of above 0.01")
print(f"There are {data[data['LactoSum'] > 0.1].shape[0]} samples that have a LactoSum value of above 0.1")

print("----------------------------------------------------------------------------------------------------")
# create binary column based on whether LactoSum is above 0.01 or not 
data['Lacto_Binary'] = np.where(data['LactoSum'] > 0.01, 1, 0)

# remove specific columns
data.drop(data.columns[lacto_index], axis=1, inplace=True) 
data.drop('LactoSum', axis=1, inplace=True)

# print count of Lacto_binary values
print(f"Lacto_Binary distribution:\n{data['Lacto_Binary'].value_counts()}")

Initial data shape: (5502, 1479)
----------------------------------------------------------------------------------------------------
There are 3399 samples that have a LactoSum value of above 0.0
There are 2726 samples that have a LactoSum value of above 0.01
There are 1366 samples that have a LactoSum value of above 0.1
----------------------------------------------------------------------------------------------------
Lacto_Binary distribution:
Lacto_Binary
0    2776
1    2726
Name: count, dtype: int64


In [152]:
data.to_csv('../../data/non_reduced/healthy_subset_df_with_lactobinary.csv', index=False)

In [153]:
# print the last column of the data frame to make sure that the Lacto_Binary column has been added
print(data.columns[-1])
# print first column to check it is the sampleID
print(data.columns[0])

Lacto_Binary
sample_id


In [154]:
X = data.iloc[:, 1:-1].values # make X all columns except the last first column (sample_id) and the last column (Lacto_Binary)
y = data.iloc[:,-1].values # make y the last column

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, # 15% of data is reserved for testing
                                                    random_state=42)

# further splitting the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, # X_train is further split into X_train and X_val
                                                  y_train, # y_train is further split into y_train and y_val
                                                  test_size=X_test.shape[0] / X_train.shape[0],  # 15% of the training set is reserved for validation
                                                  random_state=42)

In [155]:
datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test} # create a dictionary with the datasets

# open a file to write
with open('../../data/non_reduced/dataset_distribution.txt', 'w') as file:
    # print table headers and write to file
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    # loop through each dataset, print proportions and total counts, and write to file
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)  # total number of observation
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')

Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3850
Validation set  | 0.51 | 0.49 |     826
Test set        | 0.48 | 0.52 |     826


In [156]:
# save the data splits
for x, y, name in zip([X_train, X_val, X_test], 
                      [y_train, y_val, y_test], 
                      ['train', 'val', 'test']):
    combined_data = np.hstack([x, y.reshape(-1, 1)]) # combine X and y for each dataset 
    column_names = list(data.drop(columns=['sample_id', 'Lacto_Binary']).columns) + ['Lacto_Binary'] # construct list of column names, add Lacto_Binary to make sure it is the last column 
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(f'../../data/non_reduced/{name}.csv', index=False)

## 2. Prepare the reduced data for analysis (treshold at 0.1%)
As the dataset is high in dimensionality and sparsity, I take the following step to minimise noise and redundancy. This is an attempt to focus the analyses on features that are relevant. 

First, we calculate the relative abundance of each species across all samples. So for each species, we sum the relative abundance values across all observations, which gives a total abundance value for each bacterial species across all samples. Then we sum these total abundance values across all features to determine the dataset's total bacterial abundance. 

Then I set a threshold at 0.1%. So I only keep the species that make up 0.1% of the DNA mass across also samples.  Features (species) contributing less than this treshold are considered less informative/relevant for subsequent analyses and are thus candidates for removal. So each feature's total abundance is compared against the threshold. Features falling below this threshold are removed unless they are specifically associated with the Lactobacillus bacteria (as these are critical for the outcome variable of interest and are indexed in lacto_indx - but the lacto_indx cols wont be used as predictors themselves, but they'll be used to create the outcome variable).

As each sample's feature values are given as proportions of the total bacterial abundance in that sample, summing these values across all samples for each feature essentially estimates the feature's "average" presence or dominance across the dataset. So a high sum indicates a greater overall presence across the samples.

### 2.1. Across-samples threshold: Make a reduced dataset, thresholding species at 0.01% of total DNA mass across samples

In [157]:
# load the data
data = pd.read_csv('../../data/raw/healthy_subset_df.csv', index_col=0)

# extract indices for columns that include the word 'lactobacillus' (case-insensitive)
lacto_index = [i for i, s in enumerate(data.columns) if 'lactobacillus' in s.lower()]

# calculate the sum of Lactobacillus species for each row and append it to a new column 'LactoSum'
data['LactoSum'] = data.iloc[:, lacto_index].sum(axis=1)

# create a binary column based on whether LactoSum is above 0.01
data['Lacto_Binary'] = np.where(data['LactoSum'] > 0.01, 1, 0)

# remove the Lactobacillus columns and 'LactoSum' column
data.drop(data.columns[lacto_index], axis=1, inplace=True)
data.drop('LactoSum', axis=1, inplace=True)

# calculate the sum of each column (excluding 'Lacto_Binary' and 'sample_id' if it exists)
columns_to_exclude = ['Lacto_Binary']
if 'sample_id' in data.columns:
    columns_to_exclude.append('sample_id')

column_sums = data.drop(columns=columns_to_exclude).sum()

# calculate the percentage of each column sum out of the total sum
total_sum = column_sums.sum()
species_abundance = (column_sums / total_sum) * 100

# define threshold for species abundance and identify columns to keep
species_threshold = 0.01
cols_to_keep = species_abundance[species_abundance > species_threshold].index.tolist()

data_reduced = data[cols_to_keep + ['Lacto_Binary']]


print(f"Lacto_Binary distribution:\n{data_reduced['Lacto_Binary'].value_counts()}")

# save the reduced DataFrame to a CSV file
data_reduced.to_csv('../../data/reduced_0_1/healthy_subset_df_thresholded_01_with_lactobinary.csv', index=False)


Lacto_Binary distribution:
Lacto_Binary
0    2776
1    2726
Name: count, dtype: int64


In [158]:
# partition data
X = data_reduced.drop(columns=['Lacto_Binary']).values
y = data_reduced['Lacto_Binary'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Further split the training set into a training and a validation set (15% of the training set for validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=X_test.shape[0] / X_train.shape[0], random_state=42)

In [159]:
datasets = {'Training set': y_train, 'Validation set': y_val, 'Test set': y_test}

# open a file to write
with open('../../data/reduced_0_1/dataset_distribution.txt', 'w') as file:
    header = "{:<15} | {:>4} | {:>4} | {:>7}".format("Dataset", "1", "0", "Rows")
    print(header)
    file.write(header + '\n')
    
    for name, dataset in datasets.items():
        proportion = pd.Series(dataset).value_counts(normalize=True)
        total = len(dataset)
        proportion_1 = proportion.get(1, 0)
        proportion_0 = proportion.get(0, 0)
        line = "{:<15} | {:.2f} | {:.2f} | {:>7}".format(name, proportion_1, proportion_0, total)
        print(line)
        file.write(line + '\n')

Dataset         |    1 |    0 |    Rows
Training set    | 0.50 | 0.50 |    3850
Validation set  | 0.51 | 0.49 |     826
Test set        | 0.48 | 0.52 |     826


In [160]:
# save the data splits
for x, y, name in zip([X_train, X_val, X_test], [y_train, y_val, y_test], ['train', 'val', 'test']):
    combined_data = np.hstack([x, y.reshape(-1, 1)])
    
    # ensure correct column names
    column_names = list(data_reduced.drop(columns=['Lacto_Binary']).columns) + ['Lacto_Binary']
    
    # debug prints
    print(f"Shape of {name} is {combined_data.shape}, number of columns is {combined_data.shape[1]}")
    
    df = pd.DataFrame(combined_data, columns=column_names)
    df.to_csv(f'../../data/reduced_0_1/{name}.csv', index=False)

Shape of train is (3850, 278), number of columns is 278
Shape of val is (826, 278), number of columns is 278
Shape of test is (826, 278), number of columns is 278


## PCA on thresholded data

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.preprocessing import StandardScaler
plt.style.use('ggplot')

In [51]:
# file to reduced data including the Lacto_Binary variable
file = '.../../data/reduced_0_01/healthy_subset_df_threshold_0_01_Xy.csv'
df = pd.read_csv(file)
df

# define X as all columns except Lacto_Binary
X = df.drop('Lacto_Binary', axis=1)
y = df['Lacto_Binary']

FileNotFoundError: [Errno 2] No such file or directory: '/work/EmmaRisgaardOlsen#9993/microbiome-ML/data/reduced_0_01/healthy_subset_df_threshold_0_01_Xy.csv'

In [ ]:
# ------------ APPROACH 0 ----------- # https://www.kaggle.com/code/ryanholbrook/principal-component-analysis 
pca = PCA()
X_pca = pca.fit_transform(X)

# get cumulative variance explained by the components
cumvar = np.cumsum(X_pca.explained_variance_ratio_)
#Plotting cumulative variance
plt.plot(cumvar)
plt.title('Cumulative variance')
plt.xlabel('Number of components')
plt.ylabel('Variance explained')

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

X_pca.head()

loadings = pd.DataFrame(
    pca.components_.T,  # transpose the matrix of loadings
    columns=component_names,  # so the columns are the principal components
    index=X.columns,  # and the rows are the original features
)
loadings

In [ ]:
# Look at explained variance

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from sklearn.feature_selection import mutual_info_regression


def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

plot_variance(pca)

In [ ]:
# ------------ APPROACH 1 ----------- #

# Z-score the features - OBS I don't do it as vals are already standardized between 0 and 1 given that they are relative abundance
#scaler = StandardScaler() 
#scaler.fit(X)
#X = scaler.transform(X)

# The PCA model
pca = PCA(n_components=2) # estimate only 2 PCs
X_new = pca.fit_transform(X) # project the original data into the PCA spacefig, axes = plt.subplots(1,2)
axes[0].scatter(X[:,0], X[:,1], c=y)
axes[0].set_xlabel('x1')
axes[0].set_ylabel('x2')
axes[0].set_title('Before PCA')
axes[1].scatter(X_new[:,0], X_new[:,1], c=y)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].set_title('After PCA')
plt.show()

In [ ]:
print(pca.explained_variance_ratio_)